# NY Motor Vehicle Collisions – Exploratory Data Analysis  
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We look for which borough has had the maximum number of crashes reported since 2012.

- We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

  
- We look for the leading cause of crashes

- We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

- We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

- We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

- We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

- The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident.  We look for which combinations of vehicles have the most number of accidents.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [138]:
df = pd.read_pickle(r"Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


***
### Some exploratory data analysis here
***

Display information about the dataframe

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018245 entries, 0 to 2018244
Data columns (total 28 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   CRASH DATE_CRASH TIME          datetime64[ns]
 1   BOROUGH                        object        
 2   ZIP CODE                       object        
 3   LATITUDE                       float64       
 4   LONGITUDE                      float64       
 5   LOCATION                       object        
 6   ON STREET NAME                 object        
 7   CROSS STREET NAME              object        
 8   OFF STREET NAME                object        
 9   NUMBER OF PERSONS INJURED      float64       
 10  NUMBER OF PERSONS KILLED       float64       
 11  NUMBER OF PEDESTRIANS INJURED  int64         
 12  NUMBER OF PEDESTRIANS KILLED   int64         
 13  NUMBER OF CYCLIST INJURED      int64         
 14  NUMBER OF CYCLIST KILLED       int64         
 15  NUMBER OF MOTOR

## Q1 
Which borough has had the second highest total number of crashes reported since 2012? 

Count the number of cases for each borough

In [5]:
df['BOROUGH'].value_counts()

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64

In [176]:
# Display the count of cases for each borough
cases = df['BOROUGH'].value_counts()

## Q2 
Borough with the minimum number of crashes adjusted for population

We relate the number of crashes to to the borough's population to find out which borough has the minimum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website

In [177]:
# Create a DataFrame for population data
population = {'Borough':['Bronx', 'Brooklyn','Manhattan','Queens','Staten Island'],
        'Population':[1446788,2648452,1638281,2330295,487155]}
population = pd.DataFrame(bo_po)
population['Borough'] = population['Borough'].apply(str.upper)
population

,Borough,Population
0,BRONX,1446788
1,BROOKLYN,2648452
2,MANHATTAN,1638281
3,QUEENS,2330295
4,STATEN ISLAND,487155


In [178]:
cases = pd.DataFrame(cases)
# Merge the case counts DataFrame with the population DataFrame
merge_pf = pd.merge(cases, population,
         how ='inner',
         left_on='BOROUGH', right_on='Borough')
merge_pf
# Calculate the number of cases adjusted for population
merge_pf['Case_adjusted']= merge_pf['count']/merge_pf['Population']*100000
merge_pf

,count,Borough,Population,Case_adjusted
0,441026,BROOKLYN,2648452,16652.217975
1,372457,QUEENS,2330295,15983.255339
2,313266,MANHATTAN,1638281,19121.628097
3,205345,BRONX,1446788,14193.164444
4,58297,STATEN ISLAND,487155,11966.827806


## Analyzing the leading cause of crashes

Remove all rows where the contributing factor vehicle 1 is not known (ie, is 'Unspecified'), leaving only the crashes where only this information is available.

In [51]:
# Drop rows where the contributing factor for vehicle 1 is 'Unspecified'
df_cause = df.drop(df[df['CONTRIBUTING FACTOR VEHICLE 1']=='Unspecified'].index)
# Select relevant columns for analysis
df_cause[['CRASH DATE_CRASH TIME','CONTRIBUTING FACTOR VEHICLE 1']].head()

,CRASH DATE_CRASH TIME,CONTRIBUTING FACTOR VEHICLE 1
0,2021-09-11 02:39:00,Aggressive Driving/Road Rage
1,2022-03-26 11:45:00,Pavement Slippery
2,2022-06-29 06:55:00,Following Too Closely
4,2021-12-14 08:13:00,NaN
6,2021-12-14 17:05:00,Passing Too Closely


In [62]:
# Count the occurrences of each contributing factor
cause = df_cause['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()
cause

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction                           401262
Failure to Yield Right-of-Way                            119166
Following Too Closely                                    107467
Backing Unsafely                                          75042
Other Vehicular                                           62688
Passing or Lane Usage Improper                            55445
Turning Improperly                                        49908
Passing Too Closely                                       49848
Fatigued/Drowsy                                           47343
Unsafe Lane Changing                                      39711
Traffic Control Disregarded                               35210
Driver Inexperience                                       31274
Unsafe Speed                                              26915
Alcohol Involvement                                       21656
Lost Consciousness                                        20374
Reaction t

In [ ]:
len(df_cause)
sum = sum(cause)

# Calculate the proportion of each contributing factor
cause = df_cause['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()/sum
cause.head()

## Q4
Top 5 causes of crashes 

In [180]:
cause_5 = cause.head(5).sum()
cause_5

0.5803878374209062

### Q5
Considering the entire data since 2012, what is the total count of accidents that involved two or more fatalities?  (Missing values for the number of persons killed should be assumed to be equal to zero killed.)

In [82]:
# Descriptive statistics for the 'NUMBER OF PERSONS KILLED' column
df['NUMBER OF PERSONS KILLED'].describe()
# Count the number of missing values in the 'NUMBER OF PERSONS KILLED' column
df['NUMBER OF PERSONS KILLED'].isnull().sum()

31

In [83]:
# Fill missing values with 0 and recheck for missing values
df['NUMBER OF PERSONS KILLED'].fillna(0).isnull().sum()

0

In [179]:
# Updated descriptive statistics after filling missing values
df['NUMBER OF PERSONS KILLED'].describe()
# Count of accidents where two or more persons were killed
df[df['NUMBER OF PERSONS KILLED']>=2].count()

CRASH DATE_CRASH TIME            88
BOROUGH                          41
ZIP CODE                         41
LATITUDE                         76
LONGITUDE                        76
LOCATION                         76
ON STREET NAME                   75
CROSS STREET NAME                48
OFF STREET NAME                   4
NUMBER OF PERSONS INJURED        88
NUMBER OF PERSONS KILLED         88
NUMBER OF PEDESTRIANS INJURED    88
NUMBER OF PEDESTRIANS KILLED     88
NUMBER OF CYCLIST INJURED        88
NUMBER OF CYCLIST KILLED         88
NUMBER OF MOTORIST INJURED       88
NUMBER OF MOTORIST KILLED        88
CONTRIBUTING FACTOR VEHICLE 1    88
CONTRIBUTING FACTOR VEHICLE 2    51
CONTRIBUTING FACTOR VEHICLE 3    26
CONTRIBUTING FACTOR VEHICLE 4    15
CONTRIBUTING FACTOR VEHICLE 5    10
COLLISION_ID                     88
VEHICLE TYPE CODE 1              88
VEHICLE TYPE CODE 2              52
VEHICLE TYPE CODE 3              26
VEHICLE TYPE CODE 4              15
VEHICLE TYPE CODE 5         

### Q6
On average, for every 1000 accidents, how many have resulted in at least one person dead? (hint: divide the count of accidents with 1 or more deaths with the total number of accidents, and multiply by 1000)

In [181]:
# Total number of accidents
total_num = len(df)
# Total number of accidents with one or more deaths
total_death = len(df[df['NUMBER OF PERSONS KILLED'] >= 1])
# Average number of fatalities per 1000 accidents
per_death = total_death / total_num *1000
per_death

1.3893258747079764

### Q7 
What proportion of accidents in the data do not have a Cross Street Name?

In [182]:
# Proportion of accidents resulting in at least one fatality per 1000 accidents
df['CROSS STREET NAME'].isnull().sum()/len(df)

0.37435098315615795

### Q8 
The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident. Which combination of vehicles have the most number of accidents?

In [170]:
count_1 = df[['VEHICLE TYPE CODE 1','VEHICLE TYPE CODE 2']].value_counts()
count_1

VEHICLE TYPE CODE 1                  VEHICLE TYPE CODE 2                
Sedan                                Sedan                                  197944
PASSENGER VEHICLE                    PASSENGER VEHICLE                      193260
Station Wagon/Sport Utility Vehicle  Station Wagon/Sport Utility Vehicle    133780
Sedan                                Station Wagon/Sport Utility Vehicle    123915
Station Wagon/Sport Utility Vehicle  Sedan                                  123812
                                                                             ...  
Pick-up Truck                        UNKNOWN                                     1
                                     UNK L                                       1
                                     U HAU                                       1
                                     Truck                                       1
�MBU                                 Taxi                                        1
Name: count, L

In [171]:
count_2 = df[['VEHICLE TYPE CODE 2','VEHICLE TYPE CODE 1']].value_counts()
count_2

VEHICLE TYPE CODE 2                  VEHICLE TYPE CODE 1                
Sedan                                Sedan                                  197944
PASSENGER VEHICLE                    PASSENGER VEHICLE                      193260
Station Wagon/Sport Utility Vehicle  Station Wagon/Sport Utility Vehicle    133780
                                     Sedan                                  123915
Sedan                                Station Wagon/Sport Utility Vehicle    123812
                                                                             ...  
ORANGE JLG                           Van                                         1
OMT                                  Taxi                                        1
                                     OMT                                         1
                                     Bus                                         1
yw                                   Tractor Truck Diesel                        1
Name: count, L

In [174]:
pd.merge(count_1,count_2,left_on=['VEHICLE TYPE CODE 1','VEHICLE TYPE CODE 2'],right_on=['VEHICLE TYPE CODE 2','VEHICLE TYPE CODE 1'])

,count_x,count_y
0,197944,197944
1,193260,193260
2,133780,133780
3,123915,123812
4,123812,123915
...,...,...
3221,1,1
3222,1,1
3223,1,1
3224,1,1


In [185]:
# Convert columns to strings
df['VEHICLE TYPE CODE 1'] = df['VEHICLE TYPE CODE 1'].astype(str)
df['VEHICLE TYPE CODE 2'] = df['VEHICLE TYPE CODE 2'].astype(str)
# Create a new column for sorted vehicle combinations
df['SORTED VEHICLE COMBINATION'] = df.apply(lambda x: ' & '.join(sorted([x['VEHICLE TYPE CODE 1'], x['VEHICLE TYPE CODE 2']])), axis=1)

# Count occurrences of each combination
combination_counts = df['SORTED VEHICLE COMBINATION'].value_counts()
combination_counts

SORTED VEHICLE COMBINATION
Sedan & Station Wagon/Sport Utility Vehicle                                  247727
Sedan & Sedan                                                                197944
PASSENGER VEHICLE & PASSENGER VEHICLE                                        193260
Sedan & nan                                                                  138151
Station Wagon/Sport Utility Vehicle & Station Wagon/Sport Utility Vehicle    133780
                                                                              ...  
Box Truck & POST                                                                  1
Station Wagon/Sport Utility Vehicle & wagon                                       1
Van ( & nan                                                                       1
Van & trail                                                                       1
PEDICAB & nan                                                                     1
Name: count, Length: 5950, dtype: int64

### Q9
Among crashes where the contributing factor (CONTRIBUTING FACTOR VEHICLE 1) was alcohol involvement, what proportion resulted in a fatality?

(Hint: Filter your population to just those collisions where the contributing factor was Alcohol Involvement.  Then check what proportion of this population had a fatality.)

In [111]:
alcohol_df = df[df['CONTRIBUTING FACTOR VEHICLE 1']== 'Alcohol Involvement']
len(alcohol_df[alcohol_df['NUMBER OF PERSONS KILLED']!=0])/len(alcohol_df)

0.0046638345031400075

### Q10
What proportion of crashes occur during the evening rush hour, defined as starting at 4 PM, and before 7 PM?

(Hint: Find the number of crashes that occur between 4 and 7 pm using filtering.  Then divide this by the total number of crashes.  You can filter for hours in many ways, for example df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]

In [113]:
rush_df = df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]

In [114]:
len(rush_df)/len(df)

0.20514010935243243

### Q11
Among crashes involving motorcycles, what proportion resulted in injuries but no fatalities?
(Hint: Filter the dataset first to include only crashes where VEHICLE TYPE CODE 1 or VEHICLE TYPE CODE 2 contains the string MOTORCYCLE.  Then check this reduced dataset for injuries greater than 0, and fatalities equal to 0).

In [118]:
moto_df = df[(df['VEHICLE TYPE CODE 1']=='MOTORCYCLE') | (df['VEHICLE TYPE CODE 2']=='MOTORCYCLE')]

In [121]:
num_1 = len(moto_df)
num_2 = len(moto_df[(moto_df['NUMBER OF PERSONS INJURED']>0) & (moto_df['NUMBER OF PERSONS KILLED']==0)])

In [123]:
Pro = num_2/num_1
Pro

0.5004565018912221

### Q12
How many crashes involved bicycles as one of the vehicles? (Consider 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2')

In [156]:
# combine the VEHICLE TYPE CODE 1 and VEHICLE TYPE CODE 2 data with the bicycle 
bicycle_crashes = df[(df['VEHICLE TYPE CODE 1'].str.contains('BICYCLE', na=False)) | 
                     (df['VEHICLE TYPE CODE 2'].str.contains('BICYCLE', na=False))]

bicycle_crashes

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
218167,2020-10-06 16:43:00,QUEENS,11423.0,40.729210,-73.781166,"(40.72921, -73.781166)",188 STREET,UNION TURNPIKE,NaN,1.0,...,NaN,NaN,NaN,NaN,4355439,BICYCLE,NaN,NaN,NaN,NaN
1077740,2016-07-07 08:07:00,QUEENS,11373.0,NaN,NaN,NaN,BROADWAY,BAXTER AVENUE,NaN,0.0,...,Unspecified,Unspecified,NaN,NaN,3485897,BICYCLE,PASSENGER VEHICLE,BICYCLE,NaN,NaN
1092878,2016-06-17 16:06:00,BROOKLYN,11203.0,NaN,NaN,NaN,UTICA AVENUE,RUTLAND ROAD,NaN,1.0,...,Unspecified,Unspecified,Unspecified,NaN,3470666,BICYCLE,BICYCLE,NaN,NaN,NaN
1093258,2016-06-18 03:40:00,QUEENS,11105.0,40.768888,-73.906908,"(40.7688877, -73.9069078)",SOUND STREET,ASTORIA BLVD NORTH,,0.0,...,Unspecified,NaN,NaN,NaN,3463912,BICYCLE,PASSENGER VEHICLE,NaN,NaN,NaN
1144089,2016-04-05 20:27:00,NaN,NaN,NaN,NaN,NaN,FLATBUSH AVENUE,LINCOLN ROAD,,0.0,...,Unspecified,NaN,NaN,NaN,3417759,PASSENGER VEHICLE,BICYCLE,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924860,2012-07-09 17:37:00,BROOKLYN,11222,40.720601,-73.954754,"(40.7206006, -73.9547539)",BEDFORD AVENUE,NORTH 12 STREET,NaN,1.0,...,Unspecified,NaN,NaN,NaN,198192,TAXI,BICYCLE,NaN,NaN,NaN
1924881,2012-07-02 09:46:00,MANHATTAN,10002,40.717724,-73.985765,"(40.7177239, -73.9857652)",DELANCEY STREET,CLINTON STREET,NaN,1.0,...,Unspecified,NaN,NaN,NaN,12187,PICK-UP TRUCK,BICYCLE,NaN,NaN,NaN
1924949,2012-07-07 18:40:00,NaN,NaN,40.867335,-73.822707,"(40.8673349, -73.8227066)",NaN,NaN,NaN,1.0,...,Unspecified,NaN,NaN,NaN,2912116,PASSENGER VEHICLE,BICYCLE,NaN,NaN,NaN
1924950,2012-07-06 13:33:00,BROOKLYN,11209,40.625780,-74.024154,"(40.6257805, -74.0241544)",5 AVENUE,80 STREET,NaN,1.0,...,Unspecified,NaN,NaN,NaN,140835,PASSENGER VEHICLE,BICYCLE,NaN,NaN,NaN


In [160]:
# count the total number 
total_bicycle_crashes = len(bicycle_crashes)
total_bicycle_crashes

19108